# Students Performance Analysis by Region
This notebook analyzes student performance by region.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load CSV file
df = pd.read_csv(r"C:/Users/DELL/Downloads/ethiopian_students_dataset.csv")

# View first 5 rows
print(df.head())

# Access a column
print(df.columns)

In [ ]:
# 1. Get the value counts of dtypes
dtype_counts = df.dtypes.value_counts().reset_index()
dtype_counts.columns = ['Data Type', 'Count']

# 2. Plotting with the fix
plt.figure(figsize=(10, 6))

# Fix: Assign 'Data Type' to hue and set legend=False
sns.barplot(
    data=dtype_counts,
    x='Data Type',
    y='Count',
    hue='Data Type',
    palette='viridis',
    legend=False
)

plt.title('Distribution of Data Types in Student Dataset', fontsize=14)
plt.ylabel('Number of Columns')
plt.xlabel('Data Type')

# 3. Add labels on top of bars
for i, count in enumerate(dtype_counts['Count']):
    plt.text(i, count + 5, str(count), ha='center', fontweight='bold')

plt.show()

In [ ]:
#Drop ID (never use in ML)
df = df.drop(columns=['Student_ID'], errors='ignore')

#  Encode Field_Choice (VERY IMPORTANT)
df['Field_Choice'] = df['Field_Choice'].map({
    'Social': 0,
    'Natural': 1
})

# HANDLE CAREER_INTEREST
# Fill missing with "Unknown"
df['Career_Interest'] = df['Career_Interest'].fillna('Unknown')

In [ ]:
# AGGREGATE GRADES INTO EDUCATION STAGES
# Define grade groups
lower_primary = ['Grade_1', 'Grade_2', 'Grade_3', 'Grade_4']
upper_primary = ['Grade_5', 'Grade_6', 'Grade_7', 'Grade_8']
secondary = ['Grade_9', 'Grade_10']
preparatory = ['Grade_11', 'Grade_12']

# Helper function to compute average test score
def stage_average(df, grades):
    cols = []
    for g in grades:
        cols += [c for c in df.columns if c.startswith(g) and c.endswith('_Test_Score')]
    return df[cols].mean(axis=1)

# Create aggregated academic scores
df['Avg_Score_Lower_Primary'] = stage_average(df, lower_primary)
df['Avg_Score_Upper_Primary'] = stage_average(df, upper_primary)
df['Avg_Score_Secondary'] = stage_average(df, secondary)
df['Avg_Score_Preparatory'] = stage_average(df, preparatory)

# Select only the newly created columns and view the first 5 rows
textbook_summary_cols = [
    'Avg_Score_Lower_Primary',
    'Avg_Score_Upper_Primary',
    'Avg_Score_Secondary',
    'Avg_Score_Preparatory'
]

print("Aggeregated average Scores(1-4,5-8,9&10 and 11&12) per Education Level (Head):")
print(df[textbook_summary_cols].head())

import seaborn as sns
import matplotlib.pyplot as plt

# Visualizing the distribution of the four new columns
plt.figure(figsize=(10, 6))
sns.boxplot(data=df[textbook_summary_cols])
plt.title('Aggeregated average Distribution by Education Level')
plt.ylabel('Aggeregated average score')
plt.xticks(rotation=15)
plt.show()

In [ ]:
# CREATE ENGAGEMENT SCORES
# Helper function for engagement
def engagement_score(df, grades):
    attendance = []
    homework = []
    participation = []

    for g in grades:
        attendance += [c for c in df.columns if c.startswith(g) and 'Attendance' in c]
        homework += [c for c in df.columns if c.startswith(g) and 'Homework' in c]
        participation += [c for c in df.columns if c.startswith(g) and 'Participation' in c]

    return df[attendance + homework + participation].mean(axis=1)

# Create engagement features
df['Engagement_1_4'] = engagement_score(df, lower_primary)
df['Engagement_5_8'] = engagement_score(df, upper_primary)
df['Engagement_9_10'] = engagement_score(df, secondary)
df['Engagement_11_12'] = engagement_score(df, preparatory)

# -------------------------------
# Combine all Engagement scores into one overall column
# -------------------------------
engagement_cols = [
    'Engagement_1_4',
    'Engagement_5_8',
    'Engagement_9_10',
    'Engagement_11_12'
]

# Create overall engagement column
df['Engagement_All'] = df[engagement_cols].mean(axis=1)

# Check the first 5 rows
print("Overall Engagement Scores (Head):")
print(df[['Engagement_All']].head())

# Optional: Visualize the overall engagement
plt.figure(figsize=(8, 5))
sns.histplot(df['Engagement_All'], kde=True, bins=20)
plt.title('Overall Engagement Score Distribution')
plt.xlabel('Engagement Score (0 to 1)')
plt.ylabel('Count')
plt.show()

print("Engagement(attendance,homework,participation) Scores per Education Level (Head):")
print(df[engagement_cols].head())

# Visualizing the distribution of the four new columns
plt.figure(figsize=(10, 6))
sns.boxplot(data=df[engagement_cols])
plt.title('Engagement(attendance,homework,participation) Distribution by Education Level')
plt.ylabel('Average of enggegment')
plt.xticks(rotation=15)
plt.show()